## Summary

## Introduction

In [1]:
# imports

from lightgbm import LGBMRegressor
import numpy as np
import sklearn
from sklearn.model_selection import RandomizedSearchCV, KFold
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.dummy import DummyRegressor
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import (
    OneHotEncoder,
    StandardScaler,
)
from sklearn.compose import make_column_transformer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.feature_selection import RFECV
from xgboost import XGBRegressor
import vegafusion
import altair as alt
import altair_ally as aly
import seaborn as sns
import matplotlib.pyplot as plt
import shap
import pandas as pd
from scipy.stats import uniform, randint
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=DeprecationWarning)
warnings.simplefilter(action="ignore", category=UserWarning)

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


## Data

The dataset utilized for this project originates from the Kaggle, available through the following link: (https://www.kaggle.com/dgomonov/new-york-city-airbnb-open-data). It comprises 16 columns/variables and encompasses a substantial volume of data, totaling 48895 rows. The available information includes details about the properties on Airbnb, the corresponding geographical information, price , and the room type. Airbnb could use this sort of model to predict how popular future listings might be before they are posted, perhaps to help guide hosts create more appealing listings

In [10]:
df = pd.read_csv("data/AB_NYC_2019.csv", encoding="utf-8")
train_df, test_df = train_test_split(df, test_size=0.4, random_state=123)
train_df

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
17877,14010200,"Scandinavean design in Crown Heights, BK",1683437,Boram,Brooklyn,Crown Heights,40.66477,-73.95060,Entire home/apt,89,5,9,2018-05-28,0.25,1,0
14638,11563821,Private bedroom located in the heart of Chelsea,10307134,Anna,Manhattan,Chelsea,40.74118,-74.00012,Private room,110,1,48,2019-06-16,1.80,2,67
7479,5579629,Lovely sunlit room in Brooklyn,329917,Clémentine,Brooklyn,Greenpoint,40.72905,-73.95755,Private room,53,2,5,2016-10-21,0.13,1,0
47058,35575853,"Great view, 1 BR right next to Central Park!",35965489,Meygan,Manhattan,East Harlem,40.79755,-73.94797,Private room,100,2,0,NaN,NaN,1,7
9769,7509362,Great BIG Upper West Side Apartment,29156329,Andrew,Manhattan,Upper West Side,40.80120,-73.96382,Private room,87,3,9,2018-09-19,0.19,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7763,5885201,SUNNY ROOM A IN CHARMING AREA :),4291007,Graham And Ben,Brooklyn,Bedford-Stuyvesant,40.69363,-73.95980,Private room,95,30,40,2019-06-01,0.80,11,331
15377,12325045,IDEAL One bedroom apt by Central Park!,66501870,K Alexandra,Manhattan,Midtown,40.76016,-73.96910,Entire home/apt,139,2,132,2019-06-30,3.66,1,154
17730,13915004,"Sunlit, spacious NY apartment",7177483,Dani,Manhattan,Harlem,40.80380,-73.95569,Entire home/apt,250,3,10,2019-01-01,0.28,1,0
28030,21897845,One room.,159769278,Musieka,Bronx,Pelham Gardens,40.86706,-73.84674,Private room,40,2,17,2019-06-04,1.23,1,17


### EDA

In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 29337 entries, 17877 to 15725
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              29337 non-null  int64  
 1   name                            29328 non-null  object 
 2   host_id                         29337 non-null  int64  
 3   host_name                       29325 non-null  object 
 4   neighbourhood_group             29337 non-null  object 
 5   neighbourhood                   29337 non-null  object 
 6   latitude                        29337 non-null  float64
 7   longitude                       29337 non-null  float64
 8   room_type                       29337 non-null  object 
 9   price                           29337 non-null  int64  
 10  minimum_nights                  29337 non-null  int64  
 11  number_of_reviews               29337 non-null  int64  
 12  last_review                     2

In [6]:
train_df.describe().T

,count,mean,std,min,25%,50%,75%,max
id,29337.0,1.891988e+07,1.102155e+07,2539.00000,9.350729e+06,1.951751e+07,2.916531e+07,3.648561e+07
host_id,29337.0,6.714579e+07,7.835404e+07,2438.00000,7.740184e+06,3.071907e+07,1.064429e+08,2.743213e+08
latitude,29337.0,4.072901e+01,5.459419e-02,40.50641,4.069009e+01,4.072314e+01,4.076328e+01,4.091234e+01
longitude,29337.0,-7.395222e+01,4.609134e-02,-74.24442,-7.398303e+01,-7.395553e+01,-7.393643e+01,-7.371299e+01
price,29337.0,1.509391e+02,2.282242e+02,0.00000,6.900000e+01,1.070000e+02,1.750000e+02,1.000000e+04
minimum_nights,29337.0,7.141971e+00,2.227211e+01,1.00000,1.000000e+00,3.000000e+00,5.000000e+00,1.250000e+03
number_of_reviews,29337.0,2.335450e+01,4.469248e+01,0.00000,1.000000e+00,5.000000e+00,2.300000e+01,6.290000e+02
reviews_per_month,23386.0,1.369867e+00,1.706732e+00,0.01000,1.900000e-01,7.100000e-01,2.010000e+00,5.850000e+01
calculated_host_listings_count,29337.0,7.003340e+00,3.251162e+01,1.00000,1.000000e+00,1.000000e+00,2.000000e+00,3.270000e+02
availability_365,29337.0,1.128036e+02,1.315445e+02,0.00000,0.000000e+00,4.500000e+01,2.270000e+02,3.650000e+02


In [7]:
train_df.nunique()

id                                29337
name                              28894
host_id                           23964
host_name                          8375
neighbourhood_group                   5
neighbourhood                       217
latitude                          15269
longitude                         12151
room_type                             3
price                               591
minimum_nights                       94
number_of_reviews                   360
last_review                        1632
reviews_per_month                   867
calculated_host_listings_count       47
availability_365                    366
dtype: int64

### Distributions

### Distribution of categorical columns

## Preprocessing

## Baseline model and Linear model

## Different models

### Randomforest Regressor

### XGBoost Regressor

### LGBM Regressor

## Stacking

### Hyperparameter optimization

### Hyperparameter optimization and feature selection

### Interpretation and feature importances

## Conclusion

## References